# 0. Intro

# 1.


In [1]:

# oversample数据的比例
P = 0.5

N_cluster = 3


# Target data
target_path = r"../data/targets"
# target_path = r"../data/targets_5"

# Target data after DA
target_path_DA = r"../data/targets_DA"

# data keys
data_key_path = "../data/target_datakey.csv"

import os
from sklearn.cluster import KMeans
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from numpy import random
import collections
from scipy.spatial.distance import cdist
from tqdm import tqdm

In [2]:
# all_path里有全部的data地址作为list
target_all_path = os.listdir(target_path)

# 2.

In [3]:
Total_data_records = 0

In [4]:
for i in tqdm(range(len(target_all_path))):

    # Read in the data
    data_path = os.path.join(target_path,target_all_path[i])
    data = pd.read_csv(data_path,encoding="utf-8")
    data = np.array(data.iloc[:,0]).reshape(-1, 1)

    ##################### KMeans and compute dist and prob  #####################
    # Do the KMeans
    kmeans = KMeans(n_clusters=N_cluster, random_state=0).fit(data)
    centers = np.array(kmeans.cluster_centers_, dtype=int)  # 返回center
    labels =  pd.DataFrame(kmeans.labels_)                  # 返回每一点的cluster标签

    # Compute the distance from data to their center
    dist_all = cdist(data, centers, metric='cityblock')
    dist = []

    for j in range(len(dist_all)):
        dist.append(dist_all[j,kmeans.labels_[j]])
    dist = pd.DataFrame(dist)

    # Combine into one table
    dist_pd = pd.concat([pd.DataFrame(data),dist,labels],axis = 1)
    dist_pd.columns = ["N","dist","cluster"]

    # Put the 'centers position' and 'max_dist' in to the table
    centers_pd = pd.DataFrame(centers)
    centers_pd.reset_index(drop=False,inplace=True)
    centers_pd.columns = ['cluster','center']

    max_dist = pd.DataFrame(dist_pd.groupby("cluster")['dist'].max())  # 每个cluster中dist的最大值
    max_dist.reset_index(drop=False,inplace=True)
    max_dist.columns = ['cluster','max_dist']

    # Combine
    centers_pd = pd.merge(centers_pd,max_dist,on='cluster')
    data_pd = pd.merge(dist_pd,centers_pd,on='cluster',how="left")

    # Redefine the 'dist'
    data_pd['new_dist'] = data_pd['max_dist'] - data_pd['dist']+1

    # Compute the dist sum
    new_sum_dist = pd.DataFrame(data_pd.groupby("cluster")['new_dist'].sum())
    new_sum_dist.reset_index(drop=False,inplace=True)
    new_sum_dist.columns = ['cluster','new_sum_dist']

    # Combine
    data_pd = pd.merge(data_pd,new_sum_dist,on='cluster',how="left")

    # Compute the prob
    data_pd['prob'] = data_pd['new_dist']/data_pd['new_sum_dist']

    # N_news = int(P*len(data))
    # Get the amount of data to be oversampled (in each cluster)
    cluster_size = np.array(data_pd.groupby("cluster").count().iloc[:,0])
    N_news_ls = np.array(cluster_size*P,dtype=int) # 根据cluster大小分配抽样规模

    ##################### Oversample #####################

    new_data_i = pd.DataFrame()       # 新生成的data‘N’

    for j in range(kmeans.n_clusters):
        # Get all data in cluster j

        data_j = data_pd[data_pd.loc[:,'cluster'] == j]
        data_j = data_j.reset_index(drop=True)

        data_j.sort_values(by='prob',ascending = False,inplace = True,ignore_index = True) # 降序排序,并且重新index一下

        # Sample according to the prob
        new_idx = random.choice(a = data_j.shape[0], p = data_j.prob,size=N_news_ls[j])
        new_data_j = data_j.loc[new_idx,'N'].reset_index(drop=True)

        new_data_i = pd.concat([new_data_i,pd.DataFrame(new_data_j)],axis=0)

    new_data_i.reset_index(drop=True, inplace=True)
    assert new_data_i.shape[0] == N_news_ls.sum(),"Did not oversample enough data"

    ##################### Redesign the data table (Compute the 'p' value)#####################

    # Concat the old with the new
    N_data = pd.concat([pd.DataFrame(data,columns=['N']),new_data_i],axis=0,ignore_index=True)
    # Count 'cnt' to compute P
    pcount = collections.Counter(N_data.iloc[:,0])
    tmp = pd.DataFrame.from_dict(pcount,orient='index').reset_index()
    tmp.columns = ['N','cnt']
    # Compute P
    tmp['P'] = tmp.cnt/ tmp.shape[0]
    N_data = pd.merge(N_data,tmp,how='left',on=['N'])
    # Repeat data records according to 'cnt'
    output_data = N_data.loc[N_data.index.repeat(N_data['cnt'])]
    # output_data.drop(columns=['cnt'],inplace=True)

    output_path = os.path.join(target_path_DA,target_all_path[i])
    output_data.to_csv(output_path,header=True,index=False,encoding="utf-8")

print("Done")

100%|██████████| 1196/1196 [00:32<00:00, 36.25it/s]

Done
